In [1]:
import pandas as pd
from faker import Faker 
import random 
from geopy.distance import geodesic
import folium

In [7]:
#Générer des datas fictifs
fake=Faker()
Faker.seed(0)
data = []
for _ in range(1000) : 
    name = fake.name()
    age = random.randint(15, 40)
    lat = -1.67+random.uniform(-0.05, 0.05)
    lon = 29.22+random.uniform(-0.05, 0.05)
    etudiant = random.choice(["oui", "non"])
    armee = random.choice(["oui", "non"])
    casier = random.choice(["oui", "non"])

    data.append([name, age, lat, lon, etudiant, armee, casier])

df = pd.DataFrame(data, columns = ["nom", "age", "lat", "lon", "etudiant", "armee", "casier"])


In [9]:
def chercher_nom(df, nom) : 
    personne = df[df["nom"].str.contains(nom, case=False)]
    return personne

In [26]:
def filter (df, center, rayon_km, age_min= 18, age_max= 25, etudiant=None, armee=None, casier= None) : 
    resultats = []
    for _, row in df.iterrows():
        distance = geodesic(centre, (row["lat"], row["lon"])).km
        if distance <= rayon_km and age_min <= row["age"] <= age_max : 
            if (etudiant is None or row["etudiant"] == etudiant) and (armee is None or row["armee"] == armee) and (casier is None or row["casier"] == casier) :
                resultats.append(row)
    return pd.DataFrame(resultats)

centre = (-1.67, 29.22)
filter(df, centre, rayon_km=5,age_min=18, age_max=25, etudiant = "non", casier = "oui")

,nom,age,lat,lon,etudiant,armee,casier
6,Stephanie Collins,21,-1.662125,29.183195,non,oui,oui
20,Kimberly Maynard,22,-1.633107,29.220278,non,non,oui
60,Michelle Hughes,22,-1.694595,29.211669,non,non,oui
63,Jamie Hutchinson,21,-1.660985,29.177353,non,non,oui
99,Lori Bennett,22,-1.702648,29.197520,non,oui,oui
114,Erica Jimenez,24,-1.666091,29.246547,non,oui,oui
124,Kathy Santana,22,-1.627644,29.211421,non,oui,oui
153,Theresa Estrada,20,-1.663435,29.226321,non,non,oui
167,Adam Wood,22,-1.669289,29.228553,non,non,oui
187,James Saunders,25,-1.665464,29.236074,non,non,oui


In [30]:
def afficher_carte(resultats, centre, rayon_km):
    m = folium.Map(location=centre, zoom_start=13)  # zoom_start sera écrasé

    cercle = folium.Circle(
        location=centre,
        radius=rayon_km*1000,
        color="blue",
        fill=False
    ).add_to(m)
   
    #Marqueurs suspects
    points = []
    for _, row in resultats.iterrows():
        folium.Marker(
            [row["lat"], row["lon"]],
            popup=f"{row['nom']} ({row['age']} ans)",
            icon=folium.Icon(color="red", icon="remove", prefix="glyphicon")
        ).add_to(m)
        points.append([row["lat"], row["lon"]])
   
    points.append(centre)
    m.fit_bounds(points)
   
    return m

    
res = filter(df, centre, rayon_km=5, age_min=18, age_max=25, etudiant="non", casier="oui")
afficher_carte(res,centre, 5)